# White noise → PSD/ASD → band-limited $V_{rms}$ playground

This notebook generates synthetic white noise sampled at **20 kS/s**, then computes PSD/ASD using different FFT/Welch settings.

Goal: show that **band-limited RMS noise** (e.g., 100–1000 Hz) is (approximately) **invariant** to FFT parameters **when normalization + ENBW are handled correctly**.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# ---- Global parameters ----
fs = 20_000  # samples per second
duration_s = 10.0  # seconds of synthetic data
sigma_time = 1.0e-3  # time-domain RMS of noise in volts (set what you want)

band = (100.0, 1000.0)  # Hz band for Vrms integration

# Reproducibility
rng = np.random.default_rng(0)

N = int(fs * duration_s)
t = np.arange(N) / fs

# Generate white Gaussian noise with desired time-domain RMS
x = rng.normal(loc=0.0, scale=sigma_time, size=N)

print(f"Generated {duration_s:.2f} s at fs={fs} Hz → N={N} samples")
print(f"Time-domain RMS (target): {sigma_time:.3e} V")
print(f"Time-domain RMS (measured): {np.sqrt(np.mean(x**2)):.3e} V")


## Helpers

We will use `scipy.signal.welch` to estimate the **one-sided PSD** in **V²/Hz**.

Band-limited RMS is computed as:

$$V_{rms}(f_1,f_2) = \sqrt{\int_{f_1}^{f_2} S_{xx}(f)\,df}$$

Discrete approximation (Welch output):

$$V_{rms} \approx \sqrt{\sum_k S_{xx}(f_k)\,\Delta f}$$

Note: Welch already returns PSD normalized per Hz; integrating with the returned frequency spacing (which includes the estimator effects appropriately) recovers variance.


In [ ]:
def band_rms_from_psd(f, Pxx, f1, f2):
    """Compute band-limited RMS from one-sided PSD Pxx [V^2/Hz] over [f1,f2]."""
    # Select band
    m = (f >= f1) & (f <= f2)
    f_band = f[m]
    P_band = Pxx[m]
    if len(f_band) < 2:
        return np.nan
    # Integrate PSD over frequency to get variance
    var = np.trapz(P_band, f_band)
    return np.sqrt(var)

def welch_psd(x, fs, nperseg, window='hann', noverlap=None):
    f, Pxx = signal.welch(
        x,
        fs=fs,
        window=window,
        nperseg=nperseg,
        noverlap=noverlap,
        detrend='constant',
        return_onesided=True,
        scaling='density',  # PSD in V^2/Hz
        average='mean'
    )
    ASD = np.sqrt(Pxx)
    return f, Pxx, ASD


## Single run: compare Welch settings

We’ll vary `nperseg` (segment length), which changes the nominal bin spacing:

$$\Delta f \approx \frac{f_s}{n_{perseg}}$$

Even though the **spectrum curve changes**, the integrated **$V_{rms}$ in 100–1000 Hz** should stay close.


In [ ]:
settings = [
    dict(nperseg=1024,  window='hann', noverlap=512),
    dict(nperseg=2048,  window='hann', noverlap=1024),
    dict(nperseg=8192,  window='hann', noverlap=4096),
    dict(nperseg=16384, window='hann', noverlap=8192),
]

results = []

plt.figure(figsize=(9,5))
for s in settings:
    f, Pxx, ASD = welch_psd(x, fs, **s)
    vr = band_rms_from_psd(f, Pxx, *band)
    df = f[1] - f[0]
    results.append({
        'nperseg': s['nperseg'],
        'window': s['window'],
        'noverlap': s['noverlap'],
        'df_Hz': df,
        'Vrms_100_1000_V': vr,
    })
    plt.loglog(f[1:], ASD[1:])  # skip DC for log scale

plt.xlabel('Frequency (Hz)')
plt.ylabel('ASD (V/√Hz)')
plt.title('White noise ASD with different Welch nperseg (Hann)')
plt.grid(True, which='both', ls=':')
plt.show()

df_res = pd.DataFrame(results)
df_res


### Check invariance vs time-domain expectation

For ideal white noise with time-domain RMS = $\sigma$, the expected band-limited RMS is:

$$V_{rms}(f_1,f_2) \approx \sigma \sqrt{\frac{f_2-f_1}{f_s/2}}$$

This comes from the fact that (one-sided) white noise power spreads roughly uniformly from 0 to $f_s/2$.

It’s approximate (finite data, estimator variance), but gives a sanity check.


In [ ]:
sigma_meas = np.sqrt(np.mean(x**2))
f1, f2 = band
vrms_expected = sigma_meas * np.sqrt((f2 - f1) / (fs/2))

print(f"Time-domain RMS sigma = {sigma_meas:.3e} V")
print(f"Expected Vrms in {f1:.0f}-{f2:.0f} Hz (ideal white approx): {vrms_expected:.3e} V")
print("\nWelch results (from table):")
for r in results:
    print(f"  nperseg={r['nperseg']:5d}, df={r['df_Hz']:.3f} Hz → Vrms={r['Vrms_100_1000_V']:.3e} V")


## Window comparison (Rectangular vs Hann)

Welch handles normalization so that integrating PSD recovers variance.
But if you manually build PSD from FFT magnitudes, **window ENBW matters**.

Here we compare Welch with different windows while keeping the same `nperseg`.


In [ ]:
nperseg = 8192
win_settings = [
    dict(window='boxcar', noverlap=nperseg//2),
    dict(window='hann',   noverlap=nperseg//2),
    dict(window='hamming',noverlap=nperseg//2),
    dict(window='blackman',noverlap=nperseg//2),
]

plt.figure(figsize=(9,5))
rows = []
for ws in win_settings:
    f, Pxx, ASD = welch_psd(x, fs, nperseg=nperseg, **ws)
    vr = band_rms_from_psd(f, Pxx, *band)
    df = f[1]-f[0]
    rows.append({'window': ws['window'], 'df_Hz': df, 'Vrms_100_1000_V': vr})
    plt.loglog(f[1:], ASD[1:])

plt.xlabel('Frequency (Hz)')
plt.ylabel('ASD (V/√Hz)')
plt.title(f'ASD with different windows (nperseg={nperseg})')
plt.grid(True, which='both', ls=':')
plt.show()

pd.DataFrame(rows)


## Manual FFT route (for learning): PSD from FFT with window power normalization

This section shows one common **correct** way to build a one-sided PSD from a single FFT.

Key idea: scale by $f_s \sum w^2$ so that integrating PSD matches time-domain variance.


In [ ]:
def single_segment_psd_fft(xseg, fs, window='hann'):
    """One-sided PSD [V^2/Hz] from one windowed FFT segment (density scaling)."""
    N = len(xseg)
    w = signal.get_window(window, N, fftbins=True)
    xw = (xseg - np.mean(xseg)) * w
    X = np.fft.rfft(xw)
    f = np.fft.rfftfreq(N, d=1/fs)

    # Window power normalization
    U = np.sum(w**2)

    # Two-sided PSD would be |X|^2 / (fs * U)
    Pxx = (np.abs(X)**2) / (fs * U)

    # Convert to one-sided (except DC and Nyquist if present)
    if N % 2 == 0:
        # even length includes Nyquist bin at end
        Pxx[1:-1] *= 2
    else:
        Pxx[1:] *= 2

    ASD = np.sqrt(Pxx)
    return f, Pxx, ASD

# Take one segment and compare to Welch with same params
Nseg = 8192
xseg = x[:Nseg]

f1, P1, A1 = single_segment_psd_fft(xseg, fs, window='hann')
fw, Pw, Aw = welch_psd(xseg, fs, nperseg=Nseg, window='hann', noverlap=0)  # Welch on one segment

vr1 = band_rms_from_psd(f1, P1, *band)
vrw = band_rms_from_psd(fw, Pw, *band)

print(f"Manual FFT PSD Vrms(100-1000 Hz): {vr1:.3e} V")
print(f"Welch(one-seg) Vrms(100-1000 Hz):  {vrw:.3e} V")

plt.figure(figsize=(9,5))
plt.loglog(f1[1:], A1[1:])
plt.loglog(fw[1:], Aw[1:])
plt.xlabel('Frequency (Hz)')
plt.ylabel('ASD (V/√Hz)')
plt.title('Manual FFT PSD vs scipy.signal.welch (single segment)')
plt.grid(True, which='both', ls=':')
plt.show()


## Play here

Ideas:
- Change `duration_s` and see estimator variance.
- Change `sigma_time` and verify scaling.
- Change `band = (100, 1000)` to match your NEP band.
- Add a sinusoidal tone and see how it appears in ASD.


In [ ]:
# Add a tone on top of noise
f0 = 80.0
A_peak = 1e-3  # Vpeak
x_tone = x + A_peak * np.sin(2*np.pi*f0*t)

f, Pxx, ASD = welch_psd(x_tone, fs, nperseg=8192, window='hann', noverlap=4096)

plt.figure(figsize=(9,5))
plt.semilogx(f[1:], 20*np.log10(ASD[1:]))
plt.xlabel('Frequency (Hz)')
plt.ylabel('ASD (dBV/√Hz)')
plt.title('Noise + 80 Hz tone (ASD in dBV/√Hz)')
plt.grid(True, which='both', ls=':')
plt.show()

# Band-limited Vrms of noise-only vs noise+tone
vr_noise = band_rms_from_psd(*welch_psd(x, fs, nperseg=8192, window='hann', noverlap=4096)[:2], *band)
vr_tone  = band_rms_from_psd(f, Pxx, *band)
print(f"Vrms(100-1000 Hz) noise only: {vr_noise:.3e} V")
print(f"Vrms(100-1000 Hz) noise+tone:  {vr_tone:.3e} V")
